In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st

In [ ]:
coaches = pd.read_csv('combined.csv')
bengals = pd.read_csv('bengals.csv')

In [ ]:
ly=coaches.groupby(['name','level']).agg({'year':'count'})
ly_piv=ly.reset_index().pivot(index = 'name',columns = 'level',values='year')[['Coordinator','Head','Positional']]

In [ ]:
level_record = coaches[['name','level']].drop_duplicates()

In [ ]:
level_record = coaches[['name','level']].drop_duplicates()
level_record_only = (coaches.groupby(['name','level']).agg(sum).win/
                      (coaches.groupby(['name','level']).agg(sum).win + 
                       coaches.groupby(['name','level']).agg(sum).loss)).to_frame(name='wl')
lp = pd.merge(level_record,level_record_only,how='left',on = ['name','level'])
lp_pivot = lp.pivot(index='name',columns='level',values='wl')[['Coordinator','Head','Positional']]

In [ ]:
lyp = pd.merge(ly,lp,how='inner',on = ['name','level'])
lyp_piv = lyp.pivot(index = 'name', columns = 'level')

In [ ]:
byyr = (coaches.groupby(['year','phase']).agg({'name':'count'}).name/coaches.groupby('year').agg({'team':'nunique'}).team).to_frame(name='avg coaches').reset_index()
byyr_piv = byyr.pivot(index='year',columns = 'phase')['avg coaches'][['Defense','Offense','Special Teams']]

In [ ]:
cbbyyr = bengals.groupby(['year','phase']).agg({'name':'count'}).rename(columns = {'name':'coaches'}).reset_index()
cbbyyr_piv = cbbyyr.pivot(index='year',columns = 'phase')['coaches'][['Defense','Offense','Special Teams']]
allyr_piv = pd.merge(cbbyyr_piv,byyr_piv,how='right',on=['year'],suffixes=({'_bengals','_nfl'}))

In [ ]:
coach_by_yr = coaches.groupby(['year','team','phase']).agg({'name':'count'}).rename(columns={'name':'coaches'}).reset_index()
cbym = coach_by_yr.groupby(['year','phase']).agg({'coaches':'mean'}).rename(columns={'coaches':'mean'}).reset_index()
cbystd= coach_by_yr.groupby(['year','phase']).agg({'coaches':'std'}).rename(columns={'coaches':'std'}).reset_index()
tcby=pd.merge(pd.merge(cbbyyr,cbym,how = 'inner',on=['year','phase']),cbystd,how='inner',on=['year','phase'])
tcby['z']=(tcby['coaches']-tcby['mean'])/tcby['std']
tcby['p']=st.norm.cdf(tcby['z'])

In [ ]:
head_g = coaches[coaches.level == 'Head'].groupby(['name','team','level'])
head_wl = head_g.agg({'year':'count','win':'sum','loss':'sum'})
head3_ct = head_wl.loc[head_wl.year>2]
head3_ct['wl']=head_wl.win/(head_wl.win+head_wl.loss)
head3_ct = head3_ct.reset_index().rename(columns = {'year':'season'})[['name','team','wl','season']]
head3_yr = coaches[coaches.level =='Head'].groupby(['name','team','level']).agg({'year':'min'}).reset_index().rename(columns={'year':'debut'})
head3 = pd.merge(head3_ct,head3_yr,how = 'inner',on = ['name','team'])


In [ ]:
past3_m = pd.merge(head3,coaches,how='inner',on='name',suffixes=('_h','_p'))
past3_m = past3_m.loc[(past3_m.year<past3_m.debut) & (past3_m.year>past3_m.debut-4)].sort_values('year')
past3 = past3_m.groupby(['name','team_h','debut']).agg({'win':'sum','loss':'sum','position':'first',
                                                        'phase':'first','level_p':'first','team_p':'first'})
past3['wl_p']=past3.win/(past3.win+past3.loss)
past3 = past3.reset_index().rename(columns={'team_h':'team'}).drop(columns=['win','loss'])
hp3 = pd.merge(head3,past3,how='right',on=['name','team','debut'])

In [ ]:
# relationship = dict()
# names = sorted(coaches.name.dropna().unique())
# for name in names:
#     related = related_to(name)
#     relationship[name] = related


def related_to(name):
    beenhere = coaches.loc[coaches.name == name][['team','year']]
    beenhere.reset_index(inplace =True)
    firsthand = set()
    for teamyear in range(len(beenhere)):
        (i,j) = beenhere.loc[teamyear,['team','year']]
        [firsthand.add(person) for person in coaches.loc[(coaches.team == i)&(coaches.year==j)].name.tolist()]
    firsthand.remove(name)
    return firsthand 

def worked_at(name_list):
    timespace = set()
    for name in name_list:
        beenhere = coaches.loc[coaches.name == name][['team','year']]
        beenhere.reset_index(inplace = True)
        for teamyear in range(len(beenhere)):
            (i,j) = beenhere.loc[teamyear,['team','year']]
            timespace.add((i,j))
    return timespace

In [ ]:
def are_related(kevin, bacon, degree = None):
    newset=set()
    subt=set()

    if degree!=None:
        deglen=len(degree)

        degree=degree + [set()]
        timeyr = worked_at(degree[deglen-1])
        for (i,j) in timeyr:
            [newset.add(person) for person in coaches.loc[(coaches.team == i)&(coaches.year==j)].name.tolist()]

            if bacon in newset:
                print(deglen)
                print(i,' ',j)
#                 print(type(degree))
                return deglen
            else:
                degree[deglen] = degree[deglen]|newset
#                 print(len(newset), ' ',i,' ',j)
                newset=set()
            

        for i in range(0,deglen):
            subt = subt|degree[i]

        newset= newset-subt
        if newset== None:
            return 'No Connection'

        are_related(kevin, bacon, degree)
    else:
        deglen = 0
        if kevin == bacon:
            return deglen
        else:
            degree = [{kevin}]
            deglen = are_related(kevin, bacon, degree)
            return deglen




# NFL Coaches

Sources:  wikipedia.com, pro-football-reference.com

Additionally:  gb.packers.com, bengals.com

### Phases of Game

##### Offense

##### Defense

##### Special Teams

### Levels of Coaching

##### Head Coach

##### Coordinator: Offensive/Defensive Coordinators, Assistant/Associate Head Coach, Special Teams Coach

##### Positional Coaches: includes groups of positions e.g. Secondary Coach, Offensive Line Coach


## Coaches by Year

In [ ]:
plt.figure(figsize=(15,8))
no_coach = sns.lineplot(data=allyr_piv, linewidth=2.5)
no_coach.set(xlabel='Year', ylabel='Coaches/Team')
no_coach.set_title('Coaches per Team')
plt.show()

In [ ]:
tcby.loc[(tcby.p>0.95) | (tcby.p < 0.05)]

In [ ]:
lyp.groupby('level').agg({'name':'count','year':['mean','median'],'wl':['mean','median']})

## Years in Levels of Coaching

#### Coordinator vs Positional

In [ ]:

sns.set(style="white")
cvp_yr = sns.jointplot(x='Positional', y='Coordinator', data= ly_piv, kind = 'hexa')
cvp_yr.set_axis_labels(xlabel='Positional Coaching Years', ylabel='Coordinator Years')
plt.show()

#### Head Coach vs Coordinator

In [ ]:
hvc_yr = sns.jointplot(x='Coordinator', y='Head', data= ly_piv)
hvc_yr.set_axis_labels(xlabel='Coordinator Coaching Years', ylabel='Head Coaching Years')
plt.show()

#### Head Coach vs Positional

In [ ]:
hvp_yr = sns.jointplot(x='Positional', y='Head', data= ly_piv, kind = 'hexa')
sns.set(style="white")
hvp_yr.set_axis_labels(xlabel = 'Positional Coaching Years', ylabel='Head Coaching Years')
plt.show()

## Longevity at each Level

#### Positional Coach Year vs Winnig Ratio

In [ ]:
wvy_pp = sns.jointplot(x=lyp_piv.wl.Positional, y=lyp_piv.year.Positional, kind = 'hexa',xlim=(0,1), color = 'coral')
wvy_pp.set_axis_labels(xlabel = 'Positional Coaching W/L Ratios', ylabel='Positional Coaching Years')
plt.show()

#### Coordinator Year vs Winnig Ratio

In [ ]:
wvy_cc = sns.jointplot(x=lyp_piv.wl.Coordinator, y=lyp_piv.year.Coordinator, kind = 'hexa',xlim=(0,1), color = 'coral')
wvy_cc.set_axis_labels(xlabel = 'Coordinator Coaching W/L Ratios', ylabel='Coordinator Coaching Years')
plt.show()

#### Head Coach Year vs Winnig Ratio

In [ ]:
wvy_hh = sns.jointplot(x=lyp_piv.wl.Head, y=lyp_piv.year.Head, kind = 'hexa',xlim=(0,1), color = 'coral')
wvy_hh.set_axis_labels(xlabel = 'Head Coaching W/L Ratios', ylabel='Head Coaching Years')
plt.show()

## Win Ratios

#### Head Coach vs Coordinator

In [ ]:
pvc_wl = sns.jointplot(x='Positional', y='Head', data= lp_pivot,kind = 'hexa',xlim=(0,1),ylim=(0,1))
sns.set(style="white")
pvc_wl.set_axis_labels(xlabel = 'Coordinator Coaching W/L Ratios', ylabel='Head Coaching W/L Ratios')
plt.show()

#### Head Coach vs Positional Coach

In [ ]:
cvp_wl = sns.jointplot(x='Positional', y='Coordinator', data= lp_pivot,kind = 'hexa',xlim=(0,1),ylim=(0,1))
cvp_wl.set_axis_labels(xlabel = 'Positional Coaching W/L Ratios', ylabel='Head Coaching W/L Ratios')
plt.show()

#### Coordinator vs Positional Coach

In [ ]:
cvp_yrwl = sns.scatterplot(x = lyp_piv.wl.Positional, y=lyp_piv.wl.Coordinator, size=lyp_piv.year.Positional, sizes = (8,500))
cvp_yrwl.set(xlabel='Positional Coaching W/L Ratio', ylabel='Coordinator Coaching W/L Ratio',xlim=(0,1),ylim=(0,1))
cvp_yrwl.set_title('Positional Coach to Coordinator')
plt.show()

## Debuts

In [ ]:
print('Coaches lasting for 3 or more years: ', len(head3))

In [ ]:
print('Also worked in NFL for 3 or more years immediately prior: ',len(hp3))

In [ ]:
hp3_wl = sns.jointplot(x='wl_p', y='wl', data= hp3)
hp3_wl.set_axis_labels(xlabel = 'Previous Coaching W/L Ratios', ylabel='Head Coaching W/L Ratios')
plt.show()

In [ ]:
hp3_wl = sns.jointplot(x='wl_p', y='wl', data= hp3.loc[hp3.team==hp3.team_p])
hp3_wl.set_axis_labels(xlabel = 'Previous Coaching W/L Ratios', ylabel='Head Coaching W/L Ratios')
plt.show()

In [ ]:
hp3_wl = sns.jointplot(x='wl_p', y='wl', data= hp3.loc[hp3.wl>=0.5])
hp3_wl.set_axis_labels(xlabel = 'Previous Coaching W/L Ratios', ylabel='Head Coaching W/L Ratios')
plt.show()

## Further Studies

### Study Effects of Phase background

### Trace coaches from college level

### Genealogy


## Erdo Number

In [ ]:
are_related('Adam Gase','Andy Reid')

In [ ]:
(person1,person2) = coaches['name'].sample(2)
print(person1)
print(person2)
are_related(person1,person2)